# Previsão do nível de satisfação de clientes

* Acabei não utilizando o csv de test pois pelo que vi depois do treinamento, por um erro meu devia ter olhando antes, ele não tem variável target

In [1]:
import math
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

sc.setSystemProperty('spark.executor.memory', '6g')

In [2]:
spSession = SparkSession.builder.master("local").appName("SatisfacaoClienteSantander").getOrCreate()

In [3]:
trainRDD = sc.textFile("data/train.csv")
trainRDD.cache()
trainRDD.count()


76021

In [4]:
#type(trainRDD)

In [5]:
#carregando dados de teste 
#testRDD = sc.textFile("data/test.csv")
#testRDD.cache()


## Limpeza dos dados

In [6]:
trainRDD2 = trainRDD.filter(lambda x: "ID" not in x)
trainRDD2.count()

76020

In [7]:
header = trainRDD.first()

In [8]:
trainRDD2.cache()

PythonRDD[5] at RDD at PythonRDD.scala:53

In [9]:
trainRDD3 = trainRDD2.map(lambda line: list(map(float, line.split(','))))


In [10]:
trainRDD3.persist()

PythonRDD[6] at RDD at PythonRDD.scala:53

### limpeza dados de teste

In [11]:
#removendo header dados de teste
#testRDD2 = testRDD.filter(lambda x: "ID" not in x)
#testRDD2.count()

In [12]:
#testRDD3 = testRDD2.map(lambda line: list(map(float, line.split(','))))

## Análise exploratória

In [13]:
trainRDDExp = spSession.createDataFrame(trainRDD3,schema=header.split(','))

In [14]:
for i in trainRDDExp.columns:
    if not( isinstance(trainRDDExp.select(i).take(1)[0][0], str)) :
        print( "Correlação com TARGET: ", i, trainRDDExp.stat.corr('TARGET',i))

Correlação com TARGET:  ID 0.003148468722699525
Correlação com TARGET:  var3 0.004474798175536685
Correlação com TARGET:  var15 0.1013220985607169
Correlação com TARGET:  imp_ent_var16_ult1 -1.7460253766297183e-05
Correlação com TARGET:  imp_op_var39_comer_ult1 0.010353129575421496
Correlação com TARGET:  imp_op_var39_comer_ult3 0.0035169224416893973
Correlação com TARGET:  imp_op_var40_comer_ult1 0.0031193869489658296
Correlação com TARGET:  imp_op_var40_comer_ult3 -0.0003578690699807045
Correlação com TARGET:  imp_op_var40_efect_ult1 0.01922144928852695
Correlação com TARGET:  imp_op_var40_efect_ult3 0.019965206754799267
Correlação com TARGET:  imp_op_var40_ult1 0.0030872912147492
Correlação com TARGET:  imp_op_var41_comer_ult1 0.010082404008577805
Correlação com TARGET:  imp_op_var41_comer_ult3 0.0038585869783322783
Correlação com TARGET:  imp_op_var41_efect_ult1 0.02947851812252822
Correlação com TARGET:  imp_op_var41_efect_ult3 0.021485888881387284
Correlação com TARGET:  imp_op_v

Correlação com TARGET:  num_var33_0 -0.005131807460405001
Correlação com TARGET:  num_var33 -0.004917120534123466
Correlação com TARGET:  num_var34_0 -0.0010411146833006362
Correlação com TARGET:  num_var34 -0.0010411146833006362
Correlação com TARGET:  num_var35 -0.07687242112603887
Correlação com TARGET:  num_var37_med_ult2 -2.9277726026597103e-05
Correlação com TARGET:  num_var37_0 0.0016348072615708276
Correlação com TARGET:  num_var37 0.0016348072615708276
Correlação com TARGET:  num_var39_0 -0.029180745021763415
Correlação com TARGET:  num_var40_0 0.0022919814998462328
Correlação com TARGET:  num_var40 0.00975283979603897
Correlação com TARGET:  num_var41_0 -0.03030199545709127
Correlação com TARGET:  num_var41 nan
Correlação com TARGET:  num_var39 0.00975283979603897
Correlação com TARGET:  num_var42_0 -0.017944151398396436
Correlação com TARGET:  num_var42 -0.13569286688354953
Correlação com TARGET:  num_var44_0 -0.005699136506670872
Correlação com TARGET:  num_var44 -0.0050894

Correlação com TARGET:  num_var22_hace2 0.009788882808653414
Correlação com TARGET:  num_var22_hace3 -0.002856350541917358
Correlação com TARGET:  num_var22_ult1 0.0251889458610663
Correlação com TARGET:  num_var22_ult3 0.012481383583200367
Correlação com TARGET:  num_med_var22_ult3 0.016262269955026117
Correlação com TARGET:  num_med_var45_ult3 0.0020669526035314006
Correlação com TARGET:  num_meses_var5_ult3 -0.14825285370249178
Correlação com TARGET:  num_meses_var8_ult3 0.025942741941408658
Correlação com TARGET:  num_meses_var12_ult3 -0.03195846817532825
Correlação com TARGET:  num_meses_var13_corto_ult3 -0.034367326480930174
Correlação com TARGET:  num_meses_var13_largo_ult3 -0.01657692062633902
Correlação com TARGET:  num_meses_var13_medio_ult3 -0.0010411146833006325
Correlação com TARGET:  num_meses_var17_ult3 -0.005998873276223012
Correlação com TARGET:  num_meses_var29_ult3 -0.0015740440729485794
Correlação com TARGET:  num_meses_var33_ult3 -0.00491388182069379
Correlação com

###### Correlação entre as variáveis - como encontrei uma baixa correlação entre a maioria das variáveis irei componentizar pra tentar obter um melhor resultado


## Pré Processamento

In [15]:
type(trainRDD3.take(1)[0][0:-1])

list

In [16]:
def vetorizar(row) :
    obj = (row[-1], Vectors.dense(row[0:-1]))
    return obj

In [17]:
trainRDD4 = trainRDD3.map(vetorizar)

In [18]:
trainRDD4.take(1)

[(0.0,
  DenseVector([1.0, 2.0, 23.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 99.0, 0

In [19]:
trainRDD4.persist()

PythonRDD[2982] at RDD at PythonRDD.scala:53

In [20]:
#vetorizando dados de teste
#testRDD4 = testRDD3.map(vetorizar)
#testRDD4.persist()

In [21]:
trainDF = spSession.createDataFrame(trainRDD4,["label", "features"])
trainDF.select("features", "label").show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,2.0,23.0,0.0...|  0.0|
|[3.0,2.0,34.0,0.0...|  0.0|
|[4.0,2.0,23.0,0.0...|  0.0|
|[8.0,2.0,37.0,0.0...|  0.0|
|[10.0,2.0,39.0,0....|  0.0|
|[13.0,2.0,23.0,0....|  0.0|
|[14.0,2.0,27.0,0....|  0.0|
|[18.0,2.0,26.0,0....|  0.0|
|[20.0,2.0,45.0,0....|  0.0|
|[23.0,2.0,25.0,0....|  0.0|
+--------------------+-----+
only showing top 10 rows



In [22]:
#Redução de Dimensionalidade com PCA
trainPCA = PCA(k = 3, inputCol = "features", outputCol = "targetFeatures")
pcaModel = trainPCA.fit(trainDF)
pcaResult = pcaModel.transform(trainDF).select("label","targetFeatures")
pcaResult.show(truncate = False)

+-----+----------------------------------------------------------------+
|label|targetFeatures                                                  |
+-----+----------------------------------------------------------------+
|0.0  |[-0.04231956817460487,-0.074959330119039,0.03478643481687887]   |
|0.0  |[1.351394137926115,-0.09904797210653805,0.04958561039768815]    |
|0.0  |[-0.07271997667271572,-0.12875724243539194,0.059806818284099436]|
|0.0  |[-0.07027731835588574,-0.12295514120227859,0.05880821896206321] |
|0.0  |[-2.6111541071279447,-1.5647688209104584,5.4331623906853554]    |
|0.0  |[-0.09496579935822372,-0.16820545709038173,0.07806351587143814] |
|0.0  |[-0.10253873841159582,-0.18157007440862188,0.08432019301446576] |
|0.0  |[-0.2716685517727534,-0.48114329378303006,0.22334389801716237]  |
|0.0  |[-0.11006398790553293,-0.19494561685533413,0.09047549399741517] |
|0.0  |[-0.38481993135951453,-0.6815639957406224,0.3163539590882964]   |
|0.0  |[-0.2652748831298355,-0.2964187686083302,0.2

In [23]:
#indexando colunas para decision tree
stringIndexer = StringIndexer(inputCol = "label", outputCol = "indexed")
si_model = stringIndexer.fit(pcaResult)
objFinal = si_model.transform(pcaResult)
objFinal.take(1)

[Row(label=0.0, targetFeatures=DenseVector([-0.0423, -0.075, 0.0348]), indexed=0.0)]

In [24]:
#aplicando mesmo processo nos dados de teste quanto a componentização e transformação em dataframe e indexação
#testDF = spSession.createDataFrame(testRDD4,["label", "features"])
#testDF.select("features", "label").show(1)

In [25]:
#testPCA = PCA(k = 3, inputCol = "features", outputCol = "targetFeatures")
#pcaModelTest = testPCA.fit(testDF)
#pcaResultTest = pcaModelTest.transform(testDF).select("label","targetFeatures")
#pcaResultTest.show(truncate = False)

In [26]:
#stringIndexer = StringIndexer(inputCol = "label", outputCol = "indexed")
#siModelTest = stringIndexer.fit(pcaResultTest)
#objFinalTest = siModelTest.transform(pcaResultTest)
#objFinalTest.take(1)

In [27]:
objFinal.persist()
#objFinalTest.persist()

DataFrame[label: double, targetFeatures: vector, indexed: double]

## Treinamento do modelo

In [29]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = objFinal.randomSplit([0.7, 0.3])

In [30]:
#objFinalTest.count()

In [31]:
#Criando modelo de árvore de decisão 
rfClassifer = RandomForestClassifier(labelCol = "indexed", featuresCol = "targetFeatures")
modelo = rfClassifer.fit(dados_treino)

In [32]:
predictions = modelo.transform(dados_teste)
predictions.select("prediction", "indexed", "label", "targetFeatures").collect()

[Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14472674395.7778, -676917769.1916, -264551692.7232])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14472674385.7966, -676917768.7345, -264551694.6716])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14171258698.8684, -11772522000.622, -14147387.0732])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14138094818.2756, 43337671.2555, -25176050.7117])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14138094815.2603, 43337671.0058, -25176052.0304])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14138094814.7462, 43337671.3994, -25176052.2217])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14138094813.8364, 43337671.47, -25176052.6007])),
 Row(prediction=0.0, indexed=0.0, label=0.0, targetFeatures=DenseVector([-14138094813.4372, 43337671.5957, -251

In [33]:
# Avaliando a acurácia
evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "indexed", metricName = "accuracy")
evaluator.evaluate(predictions)   

0.9593218857866911

In [34]:
# Confusion Matrix
predictions.groupBy("indexed", "prediction").count().show()

+-------+----------+-----+
|indexed|prediction|count|
+-------+----------+-----+
|    1.0|       0.0|  931|
|    0.0|       0.0|21956|
+-------+----------+-----+

